
Copyright (c) 2023, University of California, Merced. All rights reserved.

This file is part of the MummyNutsBench software package developed by
the team members of Prof. Xiaoyi Lu's group (PADSYS Lab) at the University
of California, Merced.

For detailed copyright and licensing information, please refer to the license
file LICENSE in the top level directory.

In [1]:
%env LD_LIBRARY_PATH=:/home/andrew/miniconda3/envs/ssd/lib/
import tensorflow as tf
tf.test.is_gpu_available(cuda_only=True)
#import sys
#print(sys.executable)

env: LD_LIBRARY_PATH=:/home/andrew/miniconda3/envs/ssd/lib/


2022-10-23 14:09:24.320289: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-23 14:09:24.325330: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/andrew/miniconda3/envs/ssd/lib/
2022-10-23 14:09:24.325342: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2022-10-23 14:09:26.385108: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-23 14:09:28.793479: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/andrew/miniconda3/envs/ssd/lib/
2022-10-23 14:09:28.793536: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/andrew/miniconda3/envs/ssd/lib/
2022-10-23 14:09:28.793573: W tensorflow/stream_executor/platform/de

False

In [3]:
import os
CUSTOM_MODEL_NAME = 'mNuts_centerNet' 
PRETRAINED_MODEL_NAME = 'centernet_hg104_512x512_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_512x512_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'nuts_label_map.pbtxt'

paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'TRAIN_PATH': os.path.join('Tensorflow', 'workspace','images', 'train'),
    'TEST_PATH': os.path.join('Tensorflow', 'workspace','images', 'test'),
    'VALID_PATH': os.path.join('Tensorflow', 'workspace','images', 'valid'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'),
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
}

files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

#Create dir
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

env: LD_LIBRARY_PATH='/home/andrew/miniconda3/envs/ssd/lib/'


In [11]:
#Download images from roboflow (change link based on annotation set from roboflow)
!yes A | curl -L "https://app.roboflow.com/ds/5NafuLIviG?key=BjvzjqonRs" > roboflow.zip; yes y | unzip roboflow.zip -d Tensorflow/workspace/images/; rm roboflow.zip
!mv Tensorflow/workspace/images/train/Nuts_label_map.pbtxt Tensorflow/workspace/images/train/nuts_label_map.pbtxt;
!mv Tensorflow/workspace/images/train/Nuts.tfrecord Tensorflow/workspace/images/train/nuts.tfrecord;
!mv Tensorflow/workspace/images/test/Nuts_label_map.pbtxt Tensorflow/workspace/images/test/nuts_label_map.pbtxt;
!mv Tensorflow/workspace/images/test/Nuts.tfrecord Tensorflow/workspace/images/test/nuts.tfrecord;
!mv Tensorflow/workspace/images/valid/Nuts_label_map.pbtxt Tensorflow/workspace/images/valid/nuts_label_map.pbtxt;
!mv Tensorflow/workspace/images/valid/Nuts.tfrecord Tensorflow/workspace/images/valid/nuts.tfrecord;
!cp Tensorflow/workspace/images/train/nuts_label_map.pbtxt Tensorflow/workspace/annotations/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   888  100   888    0     0   3609      0 --:--:-- --:--:-- --:--:--  3609
100 2352k  100 2352k    0     0  4245k      0 --:--:-- --:--:-- --:--:-- 4245k
yes: standard output: Broken pipe
Archive:  roboflow.zip
replace Tensorflow/workspace/images/README.dataset.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:  extracting: Tensorflow/workspace/images/README.dataset.txt  
replace Tensorflow/workspace/images/README.roboflow.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:  extracting: Tensorflow/workspace/images/README.roboflow.txt  
 extracting: Tensorflow/workspace/images/test/Nuts.tfrecord  
 extracting: Tensorflow/workspace/images/test/Nuts_label_map.pbtxt  
 extracting: Tensorflow/workspace/images/train/Nuts.tfrecord  
 extracting: Tensorflow/workspace/images/train/Nuts_label_map.pbtxt  
 extracting: Tensorflow/workspace/images/valid/Nu

In [ ]:
!pip install wget
import wget

In [26]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [27]:
# Install Tensorflow Object Detection (Will need to run twice)
!apt-get install protobuf-compiler
!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 

/bin/bash: apt-get: command not found
Processing /home/andrew/mummy_nut/models/center-net/Tensorflow/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Using cached PyYAML-5.4.1-cp37-cp37m-manylinux1_x86_64.whl (636 kB)
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1695658 sha256=9bafa5a42a91f6ca8acc832587a014c76583e462202f5f3b38445c9d71d6c962
  Stored in directory: /tmp/pip-ephem-wheel-cache-phj8l5gf/wheels/f2/29/19/ba3e3051c063695da92b20e451f437d1484cbd3f54fd715b94
Successfully built object-detection
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninst

In [ ]:
# # Verify Installation
# VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# !python {VERIFICATION_SCRIPT}

In [3]:
!wget {PRETRAINED_MODEL_URL}
!mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
!cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
!cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

--2022-10-22 17:22:31--  http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_512x512_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.189.176, 2607:f8b0:4005:80c::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.189.176|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1426099846 (1.3G) [application/x-tar]
Saving to: ‘centernet_hg104_512x512_coco17_tpu-8.tar.gz’

centernet_hg104_512 100%[===================>]   1.33G   147MB/s    in 10s     

2022-10-22 17:22:42 (134 MB/s) - ‘centernet_hg104_512x512_coco17_tpu-8.tar.gz’ saved [1426099846/1426099846]

centernet_hg104_512x512_coco17_tpu-8/
centernet_hg104_512x512_coco17_tpu-8/checkpoint/
centernet_hg104_512x512_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
centernet_hg104_512x512_coco17_tpu-8/checkpoint/checkpoint
centernet_hg104_512x512_coco17_tpu-8/checkpoint/ckpt-0.index
centernet_hg104_512x512_coco17_tpu-8

In [5]:
tf.test.is_gpu_available(cuda_only=True)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2022-10-23 14:02:37.368793: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-23 14:02:37.420589: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: '/home/andrew/miniconda3/envs/ssd/lib/'
2022-10-23 14:02:37.420634: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: '/home/andrew/miniconda3/envs/ssd/lib/'
2022-10-23 14:02:37.420668: W tensorflow/stream_executor/platform/

False

In [4]:
import os
# os.environ["LD_LIBRARY_PATH"] = os.environ.get("LD_LIBRARY_PATH","")+":"+os.environ.get("CONDA_PREFIX")+"/lib"
# os.environ["LD_LIBRARY_PATH"] += ':/opt/nvidia/hpc_sdk/Linux_x86_64/21.9/cuda/11.4/targets/x86_64-linux/lib/:/opt/nvidia/hpc_sdk/Linux_x86_64/21.9/math_libs/11.4/targets/x86_64-linux/lib/:/home/andrew/centernet/cudnn-linux-x86_64-8.4.0.27_cuda11.6-archive/lib'
#os.environ["CUDA_DIR"] = '/opt/nvidia/hpc_sdk/Linux_x86_64/21.9/cuda/11.4'
#os.environ["XLA_FLAGS"] = '--xla_gpu_cuda_data_dir=/opt/nvidia/hpc_sdk/Linux_x86_64/21.9/cuda/11.4'
import object_detection
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format


In [3]:
#Setup pipeline.config
setBatchSize = '1'
pipeline = ''
with open(os.path.join(paths['CHECKPOINT_PATH'], 'pipeline.config')) as file:
  pipeline = file.read()
  pipeline = pipeline.replace("PATH_TO_BE_CONFIGURED", 'Tensorflow/workspace/pre-trained-models/centernet_hg104_512x512_coco17_tpu-8/checkpoint/ckpt-0', 1)
  pipeline = pipeline.replace('PATH_TO_BE_CONFIGURED', 'Tensorflow/workspace/images/train/nuts_label_map.pbtxt', 1)
  pipeline = pipeline.replace('PATH_TO_BE_CONFIGURED', 'Tensorflow/workspace/images/train/nuts.tfrecord', 1)
  pipeline = pipeline.replace('PATH_TO_BE_CONFIGURED', 'Tensorflow/workspace/images/valid/nuts_label_map.pbtxt', 1)
  pipeline = pipeline.replace('PATH_TO_BE_CONFIGURED', 'Tensorflow/workspace/images/valid/nuts.tfrecord', 1)
  pipeline = pipeline.replace('batch_size: 128', 'batch_size: ' + setBatchSize, 1)
  pipeline = pipeline.replace("num_classes: 90", "num_classes: 1", 1)

with open(os.path.join(paths['CHECKPOINT_PATH'], 'pipeline.config'), 'w') as file:
  file.write(pipeline)
  

In [ ]:
#fix import error
!yes y | pip uninstall opencv-python-headless
!yes y | pip install opencv-python-headless==4.1.2.30

In [9]:
!rm Tensorflow/workspace/models/mNuts_centerNet/ckpt*

rm: cannot remove 'Tensorflow/workspace/models/mNuts_centerNet/ckpt*': No such file or directory


In [14]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=3000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/mNuts_centerNet --pipeline_config_path=Tensorflow/workspace/models/mNuts_centerNet/pipeline.config --num_train_steps=3000


In [15]:
# Start Training
!{command}

2022-10-22 17:23:59.091532: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-22 17:24:05.057778: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-22 17:24:05.637063: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38236 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:17:00.0, compute capability: 8.0
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task

In [6]:

command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [7]:
# Run evaluations on newest checkpoint. Look for "checkpoint" file in mNuts_centerNet folder to change which checkpoint is being evaluated
!{command}

2022-07-08 22:19:32.468554: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
W0708 22:19:38.063506 22819421218304 model_lib_v2.py:1090] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I0708 22:19:38.063877 22819421218304 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0708 22:19:38.063936 22819421218304 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0708 22:19:38.064001 22819421218304 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0708 22:19:38.064097 22819421218304 model_lib_v2.py:1110] Expected number of evaluation epochs is 1, but instead en

In [16]:
export_script = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py')
command = f"""python {export_script} \
    --input_type image_tensor \
    --pipeline_config_path {files['PIPELINE_CONFIG']}\
    --trained_checkpoint_dir {paths["CHECKPOINT_PATH"]} \
    --output_directory ./center_net_v4
"""

!{command}


2022-10-22 18:00:21.078134: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-22 18:00:27.927815: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-22 18:00:28.502590: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38236 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:17:00.0, compute capability: 8.0
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gr

In [37]:
from object_detection.builders import model_builder

configs = config_util.get_configs_from_pipeline_file(os.path.join(paths['CHECKPOINT_PATH'], 'pipeline.config'))
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=True)

ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore("./center_net_v2/checkpoint/ckpt-0").expect_partial()


In [41]:
!wget -O inference_img.jpg 'https://drive.google.com/uc?id=1zXfzx658zrYCAgI5zJtdHnzyDlfUgbph&export=download&confirm=t' 

--2022-10-22 18:11:01--  https://drive.google.com/uc?id=1zXfzx658zrYCAgI5zJtdHnzyDlfUgbph&export=download&confirm=t
Resolving drive.google.com (drive.google.com)... 142.250.191.46, 2607:f8b0:4005:80f::200e
Connecting to drive.google.com (drive.google.com)|142.250.191.46|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-64-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/bvfmoj6a7bvntvqu18mcon6biknn4kh3/1666487400000/13385269720894965035/*/1zXfzx658zrYCAgI5zJtdHnzyDlfUgbph?e=download&uuid=07eca07e-c3b6-49db-ae85-986a2115e9fb [following]
--2022-10-22 18:11:02--  https://doc-0g-64-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/bvfmoj6a7bvntvqu18mcon6biknn4kh3/1666487400000/13385269720894965035/*/1zXfzx658zrYCAgI5zJtdHnzyDlfUgbph?e=download&uuid=07eca07e-c3b6-49db-ae85-986a2115e9fb
Resolving doc-0g-64-docs.googleusercontent.com (doc-0g-64-docs.googleusercontent.com)... 142.251.32.33, 2607:f8

In [42]:
!file inference_img.jpg

inference_img.jpg: JPEG image data, JFIF standard 1.01, aspect ratio, density 1x1, segment length 16, baseline, precision 8, 3024x4032, frames 3


In [38]:
import glob
import cv2
import numpy as np
import time
imgs = []
# names = glob.glob('../../v4_testset/*')[:64] # only take a few
names = glob.glob("./*.jpg")
for imageName in names:
    imgs.append(cv2.imread(imageName))

imgs = np.array(imgs, dtype=float)

imgs = tf.convert_to_tensor(imgs, dtype=tf.float32)


In [41]:
preprocessed_image, shapes = detection_model.preprocess(imgs)
start = time.time()
prediction_dict = detection_model.predict(preprocessed_image, shapes)
dura = time.time()-start
res = detection_model.postprocess(prediction_dict, shapes)
print(dura/len(names))

0.4559822082519531


In [12]:
scores_all = res["detection_scores"].numpy()
boxes_all = res["detection_boxes"].numpy()
for i in range(len(scores_all)):
    scores = scores_all[i]
    boxes = boxes_all[i]
    idx_np = scores >= 0.25
    content = '\n'.join(' '.join(map(str, y)) for y in boxes[idx_np])
    with open(f"../../v4_at_10/centernet/{names[i].split('/')[-1].replace('.jpg', '.txt')}", 'w') as f:
        f.write(content)

In [48]:
import numpy as np
# Source: https://pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/

# Malisiewicz et al.
def non_max_suppression_fast(boxes, overlapThresh):
	# if there are no boxes, return an empty list
	if len(boxes) == 0:
		return []
	# if the bounding boxes integers, convert them to floats --
	# this is important since we'll be doing a bunch of divisions
	if boxes.dtype.kind == "i":
		boxes = boxes.astype("float")
	# initialize the list of picked indexes	
	pick = []
	# grab the coordinates of the bounding boxes
	x1 = boxes[:,0]
	y1 = boxes[:,1]
	x2 = boxes[:,2]
	y2 = boxes[:,3]
	# compute the area of the bounding boxes and sort the bounding
	# boxes by the bottom-right y-coordinate of the bounding box
	area = (x2 - x1 + 1) * (y2 - y1 + 1)
	idxs = np.argsort(y2)
	# keep looping while some indexes still remain in the indexes
	# list
	while len(idxs) > 0:
		# grab the last index in the indexes list and add the
		# index value to the list of picked indexes
		last = len(idxs) - 1
		i = idxs[last]
		pick.append(i)
		# find the largest (x, y) coordinates for the start of
		# the bounding box and the smallest (x, y) coordinates
		# for the end of the bounding box
		xx1 = np.maximum(x1[i], x1[idxs[:last]])
		yy1 = np.maximum(y1[i], y1[idxs[:last]])
		xx2 = np.minimum(x2[i], x2[idxs[:last]])
		yy2 = np.minimum(y2[i], y2[idxs[:last]])
		# compute the width and height of the bounding box
		w = np.maximum(0, xx2 - xx1 + 1)
		h = np.maximum(0, yy2 - yy1 + 1)
		# compute the ratio of overlap
		overlap = (w * h) / area[idxs[:last]]
		# delete all indexes from the index list that have
		idxs = np.delete(idxs, np.concatenate(([last],
			np.where(overlap > overlapThresh)[0])))
	# return only the bounding boxes that were picked using the
	# integer data type
	return boxes[pick].astype("int")

# calculating iou https://stackoverflow.com/questions/25349178/calculating-percentage-of-bounding-box-overlap-for-image-detector-evaluation
def iou(b1, b2):
    if b1[0] < b1[0] or b1[1] < b1[1] or b2[0] < b2[0] or b2[1] < b2[1]:
        b1, b2 = b2, b1

    xA = max(b1[0], b2[0])
    yA = max(b1[1], b2[1])
    xB = min(b1[2], b2[2])
    yB = min(b1[3], b2[3])

    if xB < xA or yB < yA: return 0

    intersect = (yA - yB) * (xA - xB)

    A_Area = (b1[0]-b1[2])*(b1[1]-b1[3])
    B_Area = (b2[0]-b2[2])*(b2[1]-b2[3])

    return intersect/(A_Area+B_Area-intersect)

def get_map(boxes, ground_truth):
    thresholds = [0.05+i/20 for i in range(12)]
    ious = []
    precision = []
    recall = []
    for thresh in thresholds:
        FP = FN = TP = TN = 0 # true negatives are unused
        for box in boxes:
            is_fp = True # no ground truths at this location marked positive
            for truth in ground_truth:
                if iou(box, truth) >= thresh:
                    if_fp = False
                    TP += 1
            FP += is_fp
        
        for truth in ground_truth:
            is_fn = True
            for box in boxes:
                if iou(truth, box) >= thresh:
                    if_fn = False
                    break
            FN += is_fn
        # print(FN, FP, TP)
        if FP+TP == 0 or FN + TP == 0: continue
        precision.append(TP/(FP+TP))
        recall.append(TP/(FN+TP))

    recall.append(0)
    precision.append(1)
    vals = sorted([*zip(recall, precision)])
    precision = [v[1] for v in vals]
    recall = [v[0] for v in vals]

    avg_prec = sum((vals[i][1]+vals[i-1][1])*.5*(vals[i][0]+vals[i-1][0]) for i in range(1, len(vals)))/len(recall)
    return avg_prec

gt_names = glob.glob('/home/andrew/mummy_nut/datasets/2.0y/test/labels/*.txt')
gts = []

for gt_file in gt_names:
    with open(gt_file) as f:
        gts.append([[float(x) for x in y.split(' ')[1:]] for y in f])
        
print(gts)

scores_all = res["detection_scores"].numpy()
boxes_all = res["detection_boxes"].numpy()

maps = []
all_ious = np.arange(0.1, 0.5, 0.01)
for iou_thres in all_ious:
    map_for_thres = 0
    for i in range(len(scores_all)):
        scores = scores_all[i]
        boxes = boxes_all[i]
        idx_np = scores >= iou_thres
        map_for_thres+=get_map(boxes[idx_np], gts[i])
    maps.append(map_for_thres)
    print(map_for_thres)

print(all_ious[np.array(maps).argmax()])



[[[0.3192791005291005, 0.3916170634920635, 0.006283068783068783, 0.00496031746031746], [0.4384920634920635, 0.37512400793650796, 0.006613756613756613, 0.00818452380952381], [0.39021164021164023, 0.47185019841269843, 0.010582010582010581, 0.005208333333333333], [0.414021164021164, 0.6098710317460317, 0.00992063492063492, 0.011904761904761904]], [[0.626818783068783, 0.2669890873015873, 0.00958994708994709, 0.012648809523809524], [0.5783730158730159, 0.3220486111111111, 0.01455026455026455, 0.019097222222222224], [0.3996362433862434, 0.6530257936507936, 0.015542328042328041, 0.020337301587301588], [0.44345238095238093, 0.6863839285714286, 0.016534391534391533, 0.011656746031746032], [0.5186838624338624, 0.6872519841269841, 0.014219576719576719, 0.01636904761904762], [0.49867724867724866, 0.6940724206349206, 0.016534391534391533, 0.01711309523809524], [0.4730489417989418, 0.705109126984127, 0.014219576719576719, 0.017857142857142856]], [[0.5454695767195767, 0.4164186507936508, 0.0155423280